### Anotação das cadeias de correferências

In [ ]:
import pandas as pd
import pickle
import matplotlib

PATH_REDACOES = "../ccscore/data/Corpus_Redacoes_Dissertacao.pickle"

In [ ]:
# Carrega a base de redações
df_redacoes = pickle.load(open(PATH_REDACOES, 'rb'))
df_redacoes

In [ ]:
df_redacoes['Nota'].hist()

In [ ]:
df_redacoes['Competência 4'].hist()

Por meio dos valores de correlação obtidos, percebe-se que existe uma forte relação positiva
entre a Nota Final e o valor da Competência 4.
Já os valores de relação entre a Competência 4 e as demais, mostra que existe uma relação positiva
moderada entre as Competências, o que faz com que exista uma influência equivalente entre a Competência 4
e todas as outras competências. 

In [ ]:
df = pd.DataFrame(df_redacoes[['Nota', 
                               'Competência 1',
                               'Competência 2',
                               'Competência 3',
                               'Competência 4',
                               'Competência 5']])
correlation = df.corr(method='pearson')
correlation


Pelo teste t de Student feito a seguir, podemos ver que pelo do p-valor ser 0 (zero),
a relação entre a Competência 4 e a Nota Final é estatisticamente significante. Enquanto, 
o relacionamento entre a Competência 4 e as demais, pode ser considerado estatisticamente 
significante, mas que o mesmo 

In [ ]:
import numpy as np
from scipy.stats import ttest_ind

nota = df_redacoes[['Nota']].to_numpy()

compt_1 = df_redacoes[['Competência 1']].to_numpy()
compt_2 = df_redacoes[['Competência 2']].to_numpy()
compt_3 = df_redacoes[['Competência 3']].to_numpy()
compt_4 = df_redacoes[['Competência 4']].to_numpy()
compt_5 = df_redacoes[['Competência 5']].to_numpy()

res = ttest_ind(nota, compt_1)
print("Nota, Competência 1: ",res)

res = ttest_ind(nota, compt_2)
print("Nota, Competência 2: ",res)

res = ttest_ind(nota, compt_3)
print("Nota, Competência 3: ",res)

res = ttest_ind(nota, compt_4)
print("Nota, Competência 4: ",res)

res = ttest_ind(nota, compt_5)
print("Nota, Competência 5: ",res)

res = ttest_ind(compt_1, compt_4)
print("Competência 1, Competência 4: ",res)

res = ttest_ind(compt_2, compt_4)
print("Competência 2, Competência 4: ",res)

res = ttest_ind(compt_3, compt_4)
print("Competência 3, Competência 4: ",res)

res = ttest_ind(compt_5, compt_4)
print("Competência 5, Competência 4: ",res)


In [ ]:
from sklearn.preprocessing import MinMaxScaler

df = df_redacoes[['Nota', 'Competência 4']]
min_max_scaler = MinMaxScaler()
df[['Nota', 'Competência 4']] = min_max_scaler.fit_transform(df[['Nota', 'Competência 4']])

df.sort_values(by=['Competência 4'], inplace=True)

lines = df.reset_index().plot.line(y='Competência 4', x='index', c='Darkblue')

#df['Nota'].value_counts().hist()
#df['Competência 4'].value_counts().hist()
#lines = df.reset_index().plot.scatter(x='Competência 4', y='Nota', c='Darkblue')
#df_contagem = df['Nota'].value_counts()
#df_contagem = pd.concat([df_contagem, df['Competência 4'].value_counts()], axis=0)
#df_contagem

In [ ]:
# Contagem de palavras das redacoes
total_palavras = 0

for i,redacao in df_redacao.iterrows():
    texto = redacao['Texto']
    texto = texto.replace("\n", ' ')

    doc = nlp(texto)
    total_palavras += len([token.text for token in doc if token.is_punct != True])

total_palavras

In [ ]:
print(f"Total de palavras de todas as redações: {total_palavras}")
print(f"Média de palavras por redação: {round(total_palavras/total_redacoes)}")

In [ ]:
#redacao_exemplo = nlp(df_redacao[["Texto"]][0])
i_redacao = 10
print(df_redacao["Texto"][i_redacao])
df_redacao.iloc[i_redacao]

In [ ]:
# Grava os arquivos as redações em arquivos para anotação
# utilizando o software CORP

for i,redacao in df_redacao.iterrows():
    texto = redacao['Texto']
    arq_redacao = f'../CORP/Entrada/redacao_{i}.txt'
    try:
        with open(arq_redacao, 'w') as f:
            f.write(texto)
    except IOError as e:
        print(f"Erro ao processar o arquivo {arq_redacao}")


In [ ]:
# Carga das redacoes 

import re

# Corrige os textos dos arquivos, alterando
# principalmente, situações de pontuação 
# sem espaço subsequente
def ajusta_texto(texto):
    
    pats = [r"\..[A-Za-z^\s]",   # ponto "." sem espaço subsequente
            r",.[A-Za-z^\s]",    # virgula "," sem espaço subsequente
            r"\(.[A-Za-z^\s]",    # parentese "(" sem espaço antecedente
            r".[A-Za-z^\s]\)"    # parentese "(" sem espaço subsequente
           ]
    chars = [".", ",", "(", ")"]
     
    for c, pat in zip(chars, pats):
        matches = re.findall(pat, texto, re.MULTILINE)         

        for m in matches:
            pos_m = texto.find(m)
            pos_dot = m.find(c)
            
            if c == "(":
                if not(texto[pos_m-1] == " "):
                    m_rep =  f" {m}"
                    texto = texto.replace(m, m_rep)
            else:
                pos_m = pos_m + pos_dot
                if ((pos_m+1) < len(texto)):
                    if not(texto[pos_m+1] == " "):
                        if c == ',' and texto[pos_m+1] in "0123456789":
                            continue
                        m_rep = f"{m[:pos_dot+1]} {m[pos_dot+1:]}"
                        texto = texto.replace(m, m_rep)
    
    texto = texto.replace("\'\'", "\"")
    texto = texto.replace(" .", ".")
    texto = texto.replace(" ,", ",")
    
    return texto

In [ ]:
print(texto_redacao)
print(100*"=")
print(ajusta_texto(texto_redacao))

In [ ]:
s1 = "1,23 milhões ver sem amigos , pois faz. parte(ou mais parte),ou não faria e,outra vez(quem sabe) . "
ajusta_texto(s1)

In [ ]:
# Gerando um arquivo contendo todas as redações
PATH_ALL = "./redacoes/"
l_separador = "\n"

for arq_essay in arqs_essays:
    texto = ""
    texto += f"<ARQUIVO: \"{arq_essay}\">"
    texto += l_separador    
    with open(arq_essay, "r") as f_essay:
        texto += ajusta_texto("".join(f_essay.readlines()))
    
    texto += l_separador        
    with open(f"{PATH_ALL}all_texts.txt", "a") as f_todos:
        f_todos.write(texto)
       
    
    

### Avaliando os erros ortograficos presentes nos arquivos

In [ ]:
import language_tool_python as ltp

tool = ltp.LanguageTool('pt-BR')


In [ ]:
path_redacao = arqs_essays[25]

texto_redacao = ""
with open(path_redacao) as f:
    texto_redacao = "".join(f.readlines())

texto_redacao 
erros = tool.check(ajusta_texto(texto_redacao))

print(texto_redacao)
print()
print(str(erros).replace('), Match', "), \n\nMatch)"))
#print("\n\n".join([str(x) for x in erros]))